# Notebook designed to be run in colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
# Change directory to the specified path
os.chdir('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution')
# Verify the current working directory
print("Current working directory:", os.getcwd())
!pip install -r requirements.txt
import sys
sys.path.append('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution')
!pip install sacremoses
!pip install sounddevice scipy openai-whisper

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.autograd import Variable
import pickle
# Ensure Python can find your modules by adding directories to the path
sys.path.append('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution')
sys.path.append('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution/src')
sys.path.append('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution/src/models')
sys.path.append('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution/src/services')
sys.path.append('/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution/src/utils')
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory: /content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/saiCommandExecution
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pytest-8.2.2-py3-none-any.whl (339 kB)
  Using cached scipy-1.13.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
  error: subprocess-exited-with-error
  
  × Building wheel for PyAudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a

In [2]:
import importlib
# Import your modules
import models.cnet
import services.cnet_inference
import utils.dataloader

# Reload the modules to ensure the latest changes are picked up
importlib.reload(models.cnet)
importlib.reload(services.cnet_inference)
importlib.reload(utils.dataloader)

# Ensure that the BertLayer class is available
from models.cnet import BertLayer, Encoder, Middle, Decoder, PositionalEncoding
from src.utils.dataloader import tokenize_sample
from src.services.cnet_inference import predict_intent_and_tags, run_inference

# Now use the reloaded modules
from models import cnet
from services import cnet_inference
from utils import dataloader


In [3]:
!apt-get install -y ffmpeg
!pip install pydub scipy openai-whisper

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


# main test

In [7]:
# import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write
import whisper
import IPython.display as ipd
import torch
from pydub import AudioSegment
from pydub.playback import play
from google.colab import output
import base64
import time

def record_audio(duration=5, sample_rate=16000):
    # Record audio using Google Colab's built-in recorder
    print("Recording audio...")
    record_js = """
    const sleep = time => new Promise(resolve => setTimeout(resolve, time))
    const b2text = blob => new Promise(resolve => {
        const reader = new FileReader()
        reader.onload = () => resolve(reader.result)
        reader.readAsDataURL(blob)
    })

    var record = time => new Promise(resolve => {
        navigator.mediaDevices.getUserMedia({ audio: true }).then(stream => {
            recorder = new MediaRecorder(stream)
            chunks = []
            recorder.ondataavailable = e => chunks.push(e.data)
            recorder.start()
            sleep(time).then(() => {
                recorder.onstop = async () => {
                    blob = new Blob(chunks)
                    text = await b2text(blob)
                    resolve(text)
                }
                recorder.stop()
            })
        })
    })

    """
    display(ipd.Javascript(record_js))
    audio_data = output.eval_js('record(%d)' % (duration * 1000))
    audio_file_path = "/content/temp_audio.wav"
    with open(audio_file_path, "wb") as f:
        f.write(base64.b64decode(audio_data.split(",")[1]))
    return audio_file_path

def main():

    try:
        print("Listening...")
        duration = 5  # Adjust duration as needed

        # Record audio from the microphone
        audio_file_path = record_audio(duration=duration)

        # Play the recorded audio
        display(ipd.Audio(audio_file_path))

        # Process the audio using Whisper
        print("Processing audio with Whisper...")
        start_time = time.time()
        result = whisper_model.transcribe(audio_file_path)
        end_time = time.time()
        transcription_time = end_time - start_time
        user_speech = result["text"]
        print("Heard:", user_speech)
        print(f"Transcription took {transcription_time:.2f} seconds")
        if user_speech:
            closest_command = run_inference(user_speech, '/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/bert-large-uncased', model)
            print("Interpreted command:", closest_command)
    except KeyboardInterrupt:
        print("Stopping...")

# Ensure you have the necessary packages installed
# !apt-get install -y ffmpeg
# !pip install pydub scipy openai-whisper

# Run the main function
model_path = "/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/models/ctranfinal_oop_length_v3"
model = cnet.CNet(model_path=model_path)
model = model.cuda()

# Load the Whisper model
whisper_model = whisper.load_model("base")

MIDDLE 60
DECODER 60


In [9]:
main()


Listening...
Recording audio...


<IPython.core.display.Javascript object>

Processing audio with Whisper...
Heard:  A text John and telling his order is ready at McDonald's.
Transcription took 0.55 seconds
example input:BOS a text john and telling his order is ready at mcdonald s EOS	 O O O O O O O O O O O O O open_app
Interpreted command: {'intent': 'send_text', 'tags': [('john', 'B-contact_name'), ('mcdonald', 'B-contact_name'), ('s', 'I-contact_name')], 'timings': {'bert_input_prep': 0.0003781318664550781, 'sequence_prep': 0.00010228157043457031, 'model_processing': 0.26383471488952637}}


# separate instances

In [ ]:

model_path="/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/models/ctranfinal_oop_length_v3"  # file unique id for saving and loading models
model = cnet.CNet(model_path=model_path)
model = model.cuda()




# Inference

In [ ]:
print(run_inference("play the beatles on apple music", '/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/bert-large-uncased', model))


In [ ]:
print(run_inference("start a one hour timer and name it shabadoo", '/content/drive/MyDrive/SoftAcuity Models/speechCommands/CTran-main/bert-large-uncased', model))
